# Bài tập thực hành 1.3.3: Tìm hiểu các tính năng và cách sử dụng sản phẩm AutoViz

## 3.1 Giới thiệu

### AutoViz là một thư viện Python giúp tự động hóa việc trực quan hóa dữ liệu. Nó tạo nhiều loại biểu đồ để phân tích mối quan hệ giữa các biến, phân phối dữ liệu, và các đặc tính quan trọng khác.

## 3.2 Cài đặt

In [7]:
# Cài đặt bằng lệnh:
# pip install autoviz

## 3.3 Cách sử dụng

In [8]:
import pandas as pd
import numpy as np
from autoviz.AutoViz_Class import AutoViz_Class
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

In [9]:
print("="*80)
print("PHÂN TÍCH DỮ LIỆU MARKETING CAMPAIGN SỬ DỤNG AUTOVIZ")
print("="*80)

# =============================================================================
# BƯỚC 1: ĐỌC VÀ XỬ LÝ DỮ LIỆU
# =============================================================================
print("\n" + "="*80)
print("BƯỚC 1: ĐỌC VÀ XỬ LÝ DỮ LIỆU")
print("="*80)

# Đọc dữ liệu
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')
print(f"\n✓ Đã đọc dữ liệu: {df.shape[0]} dòng, {df.shape[1]} cột")
print(f"\nCác cột trong dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

# Xử lý dữ liệu
print("\n" + "-"*80)
print("Xử lý dữ liệu...")
print("-"*80)

# 1. Xử lý giá trị null trong Income
null_count = df['Income'].isnull().sum()
if null_count > 0:
    median_income = df['Income'].median()
    df['Income'].fillna(median_income, inplace=True)
    print(f"✓ Đã điền {null_count} giá trị null trong Income bằng median: ${median_income:,.2f}")

# 2. Chuyển đổi Dt_Customer sang datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
current_date = df['Dt_Customer'].max()
df['Customer_Days'] = (current_date - df['Dt_Customer']).dt.days
df['Customer_Years'] = df['Customer_Days'] / 365.25
print("✓ Đã chuyển đổi Dt_Customer và tạo Customer_Days, Customer_Years")

# 3. Tính tuổi
df['Age'] = 2014 - df['Year_Birth']
print("✓ Đã tính tuổi từ Year_Birth")

# 4. Loại bỏ outliers trong Age
original_count = len(df)
df = df[(df['Age'] > 18) & (df['Age'] < 100)]
removed = original_count - len(df)
if removed > 0:
    print(f"✓ Đã loại bỏ {removed} outliers trong Age")

# 5. Tạo các feature tổng hợp
df['Total_Purchases'] = (df['NumWebPurchases'] + df['NumCatalogPurchases'] + 
                         df['NumStorePurchases'] + df['NumDealsPurchases'])

df['Total_Amount_Spent'] = (df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + 
                            df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds'])

df['Total_Accepted_Campaigns'] = (df['AcceptedCmp1'] + df['AcceptedCmp2'] + 
                                   df['AcceptedCmp3'] + df['AcceptedCmp4'] + 
                                   df['AcceptedCmp5'] + df['Response'])

df['Total_Children'] = df['Kidhome'] + df['Teenhome']

print("Đã tạo các feature: Total_Purchases, Total_Amount_Spent, Total_Accepted_Campaigns, Total_Children")

# 6. Tạo segment theo chi tiêu
df['Spending_Segment'] = pd.cut(df['Total_Amount_Spent'], 
                                 bins=[0, 100, 500, 1000, float('inf')],
                                 labels=['Low', 'Medium', 'High', 'Premium'])
print("Đã tạo Spending_Segment (Low, Medium, High, Premium)")

print(f"\nKích thước dữ liệu sau xử lý: {df.shape[0]} dòng, {df.shape[1]} cột")

# Lưu dữ liệu đã xử lý
df.to_csv('marketing_campaign_processed.csv', index=False)
print("Đã lưu dữ liệu đã xử lý vào 'marketing_campaign_processed.csv'")

PHÂN TÍCH DỮ LIỆU MARKETING CAMPAIGN SỬ DỤNG AUTOVIZ

BƯỚC 1: ĐỌC VÀ XỬ LÝ DỮ LIỆU

✓ Đã đọc dữ liệu: 2240 dòng, 29 cột

Các cột trong dataset:
   1. ID
   2. Year_Birth
   3. Education
   4. Marital_Status
   5. Income
   6. Kidhome
   7. Teenhome
   8. Dt_Customer
   9. Recency
  10. MntWines
  11. MntFruits
  12. MntMeatProducts
  13. MntFishProducts
  14. MntSweetProducts
  15. MntGoldProds
  16. NumDealsPurchases
  17. NumWebPurchases
  18. NumCatalogPurchases
  19. NumStorePurchases
  20. NumWebVisitsMonth
  21. AcceptedCmp3
  22. AcceptedCmp4
  23. AcceptedCmp5
  24. AcceptedCmp1
  25. AcceptedCmp2
  26. Complain
  27. Z_CostContact
  28. Z_Revenue
  29. Response

--------------------------------------------------------------------------------
Xử lý dữ liệu...
--------------------------------------------------------------------------------
✓ Đã điền 24 giá trị null trong Income bằng median: $51,381.50
✓ Đã chuyển đổi Dt_Customer và tạo Customer_Days, Customer_Years
✓ Đã tính tuổ

In [10]:
# =============================================================================
# BƯỚC 2: PHÂN TÍCH VỚI AUTOVIZ
# =============================================================================
print("\n" + "="*80)
print("BƯỚC 2: PHÂN TÍCH VỚI AUTOVIZ")
print("="*80)

print("\n📊 AutoViz là gì?")
print("-"*80)
print("""
AutoViz (Automatic Visualization) là một thư viện Python mạnh mẽ giúp:
• Tự động tạo các biểu đồ phân tích dữ liệu
• Phát hiện mối quan hệ giữa các biến
• Hiển thị phân phối dữ liệu
• Tìm outliers và patterns
• Tiết kiệm thời gian phân tích khám phá (EDA)

Các loại biểu đồ AutoViz tạo ra:
1. Scatter plots - Phân tích mối quan hệ giữa 2 biến liên tục
2. Distribution plots - Phân phối của từng biến
3. Violin plots - Phân phối theo nhóm categorical
4. Heatmap - Ma trận tương quan
5. Bar plots - So sánh categorical variables
""")

print("\n" + "-"*80)
print("Chuẩn bị dữ liệu cho AutoViz...")
print("-"*80)

# =============================================================================
# CHUẨN BỊ DỮ LIỆU
# =============================================================================
import os
import numpy as np

# Chọn các cột quan trọng để visualize
cols_to_viz = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_remove = ['ID', 'Z_CostContact', 'Z_Revenue', 'Year_Birth']
cols_to_viz = [col for col in cols_to_viz if col not in cols_to_remove]

print(f"\n✓ Đã chọn {len(cols_to_viz)} biến số để phân tích:")
for i, col in enumerate(cols_to_viz, 1):
    print(f"  {i:2d}. {col}")

categorical_cols = ['Education', 'Marital_Status', 'Spending_Segment']
print(f"\n✓ Đã chọn {len(categorical_cols)} biến categorical:")
for i, col in enumerate(categorical_cols, 1):
    print(f"  {i}. {col}")

df_viz = df[cols_to_viz + categorical_cols].copy()
print(f"\n✓ DataFrame cho AutoViz: {df_viz.shape[0]} dòng, {df_viz.shape[1]} cột")

# =============================================================================
# CHẠY AUTOVIZ
# =============================================================================
print("\n" + "-"*80)
print("🚀 Đang chạy AutoViz...")
print("-"*80)


from autoviz.AutoViz_Class import AutoViz_Class
from IPython.display import Image, display
import glob

save_dir = os.path.join(os.getcwd(), "AutoViz_Plots")
os.makedirs(save_dir, exist_ok=True)

# Khởi tạo AutoViz
AV = AutoViz_Class()

try:
    dft = AV.AutoViz(
        filename='',                        
        dfte=df_viz,                        
        depVar='Total_Amount_Spent',        
        verbose=1,                          
        lowess=False,                       
        chart_format='svg',                 
        max_rows_analyzed=2000,             
        max_cols_analyzed=30,
        save_plot_dir=save_dir              
    )
    
    print("\n" + "="*80)
    print("✅ AUTOVIZ ĐÃ HOÀN THÀNH!")
    print("="*80)
    print(f"\n📁 Các biểu đồ đã được lưu trong thư mục: {save_dir}")


except Exception as e:
    print(f"\n❌ LỖI KHI CHẠY AUTOVIZ: {e}")
    print("\nGợi ý khắc phục:")
    print("1. Kiểm tra lại cài đặt: pip install autoviz")
    print("2. Thử chạy với dữ liệu nhỏ hơn")
    print("3. Kiểm tra version: pip show autoviz")



BƯỚC 2: PHÂN TÍCH VỚI AUTOVIZ

📊 AutoViz là gì?
--------------------------------------------------------------------------------

AutoViz (Automatic Visualization) là một thư viện Python mạnh mẽ giúp:
• Tự động tạo các biểu đồ phân tích dữ liệu
• Phát hiện mối quan hệ giữa các biến
• Hiển thị phân phối dữ liệu
• Tìm outliers và patterns
• Tiết kiệm thời gian phân tích khám phá (EDA)

Các loại biểu đồ AutoViz tạo ra:
1. Scatter plots - Phân tích mối quan hệ giữa 2 biến liên tục
2. Distribution plots - Phân phối của từng biến
3. Violin plots - Phân phối theo nhóm categorical
4. Heatmap - Ma trận tương quan
5. Bar plots - So sánh categorical variables


--------------------------------------------------------------------------------
Chuẩn bị dữ liệu cho AutoViz...
--------------------------------------------------------------------------------

✓ Đã chọn 29 biến số để phân tích:
   1. Income
   2. Kidhome
   3. Teenhome
   4. Recency
   5. MntWines
   6. MntFruits
   7. MntMeatProducts
 

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Income,float64,0.000000,NA,1730.000000,666666.000000,Column has 6 outliers greater than upper bound (116863.00) or lower than lower bound(-12689.00). Cap them or remove them.
Customer_Years,float64,0.000000,NA,0.000000,1.913758,No issue
Kidhome,int64,0.000000,0,0.000000,2.000000,No issue
Teenhome,int64,0.000000,0,0.000000,2.000000,No issue
Recency,int64,0.000000,5,0.000000,99.000000,No issue
MntWines,int64,0.000000,39,0.000000,1493.000000,Column has 29 outliers greater than upper bound (1226.50) or lower than lower bound(-697.50). Cap them or remove them.
MntFruits,int64,0.000000,8,0.000000,199.000000,Column has 189 outliers greater than upper bound (81.00) or lower than lower bound(-47.00). Cap them or remove them.
MntMeatProducts,int64,0.000000,28,0.000000,1622.000000,Column has 159 outliers greater than upper bound (541.00) or lower than lower bound(-299.00). Cap them or remove them.
MntFishProducts,int64,0.000000,9,0.000000,259.000000,Column has 183 outliers greater than upper bound (119.50) or lower than lower bound(-68.50). Cap them or remove them.
MntSweetProducts,int64,0.000000,9,0.000000,263.000000,Column has 200 outliers greater than upper bound (83.50) or lower than lower bound(-48.50). Cap them or remove them.


Number of All Scatter Plots = 3
All Plots done
Time to run AutoViz = 5 seconds 

 ###################### AUTO VISUALIZATION Completed ########################

✅ AUTOVIZ ĐÃ HOÀN THÀNH!

📁 Các biểu đồ đã được lưu trong thư mục: d:\HK1_2025-2026\Phân tích dữ liệu\PTDL_2025\BaiTapLon1\BaiTapLon1_Nhom\1.3.4\AutoViz_Plots


In [11]:
# =============================================================================
# BƯỚC 3: GIẢI THÍCH CÁC BIỂU ĐỒ AUTOVIZ TẠO RA
# =============================================================================
print("\n" + "="*80)
print("BƯỚC 3: GIẢI THÍCH CÁC LOẠI BIỂU ĐỒ AUTOVIZ TẠO RA")
print("="*80)

explanation = """
AutoViz tự động tạo nhiều loại biểu đồ khác nhau, được tổ chức theo các nhóm:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 1. SCATTER PLOTS (Biểu đồ phân tán)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• Hiển thị mối quan hệ giữa 2 biến liên tục
• Xác định correlation (tương quan)
• Phát hiện outliers và patterns

Cách đọc:
• Mỗi điểm = 1 khách hàng
• Trục X: Biến độc lập (ví dụ: Income)
• Trục Y: Total_Amount_Spent (biến mục tiêu)
• Màu sắc: Phân biệt theo nhóm (nếu có)

Ví dụ biểu đồ AutoViz tạo:
• Income vs Total_Amount_Spent
  → Nếu các điểm tạo thành đường thẳng đi lên: Correlation dương
  → Income cao → Chi tiêu cao
  
• Age vs Total_Amount_Spent
  → Nếu các điểm rải rác: Không có correlation rõ ràng
  → Tuổi không ảnh hưởng nhiều đến chi tiêu

• Total_Purchases vs Total_Amount_Spent
  → Mua nhiều lần → Chi tiêu tổng cao
  → Quan trọng cho chiến lược loyalty

💡 INSIGHT: Tìm các biến có correlation mạnh để:
   - Targeting customers
   - Predictive modeling
   - Segmentation

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📈 2. DISTRIBUTION PLOTS (Biểu đồ phân phối)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• Hiển thị cách dữ liệu được phân bố
• Xác định skewness (độ lệch)
• Tìm outliers

Loại biểu đồ:
a) Histogram:
   • Trục X: Giá trị của biến
   • Trục Y: Tần suất (số lượng)
   • Hình dạng cho biết distribution type

b) KDE Plot (Kernel Density Estimate):
   • Đường cong mượt mà thể hiện density
   • Peak = giá trị phổ biến nhất

Ví dụ với Total_Amount_Spent:
• Hình dạng: Right-skewed (lệch phải)
  → Phần lớn khách chi tiêu thấp
  → Một số ít chi tiêu rất cao (VIP)

• Mean > Median
  → Có outliers ở phía cao
  → Top customers kéo trung bình lên

💡 INSIGHT:
   - Nếu right-skewed: Focus vào top customers
   - Nếu normal distribution: Mass market strategy
   - Outliers = VIP customers cần retention

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎻 3. VIOLIN PLOTS (Biểu đồ Violin)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• So sánh phân phối giữa các nhóm categorical
• Kết hợp box plot và density plot

Cách đọc:
• Chiều rộng = Density (nhiều data points)
• Chiều dài = Phạm vi giá trị
• Đường giữa = Median

Ví dụ: Total_Amount_Spent theo Education
• PhD: Violin hẹp ở trên → Chi tiêu tập trung ở mức cao
• Basic: Violin rộng ở dưới → Chi tiêu tập trung ở mức thấp
• Graduation: Violin rộng cả 2 đầu → Đa dạng

💡 INSIGHT:
   - Nhóm nào có spending cao và ổn định?
   - Nhóm nào có potential để upsell?
   - Nên target nhóm nào?

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔥 4. HEATMAP - MA TRẬN TƯƠNG QUAN
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• Hiển thị correlation giữa TẤT CẢ các biến số
• Tìm multicollinearity
• Xác định biến quan trọng

Cách đọc:
• Màu đỏ đậm: Correlation dương mạnh (+0.7 đến +1.0)
• Màu xanh đậm: Correlation âm mạnh (-0.7 đến -1.0)
• Màu trắng: Không correlation (0)

Ví dụ với Total_Amount_Spent:
• Income: +0.8 (đỏ đậm) → Quan trọng nhất
• Total_Purchases: +0.7 (đỏ) → Rất quan trọng
• Total_Children: -0.3 (xanh nhạt) → Có con = chi tiêu giảm
• Age: ~0 (trắng) → Không ảnh hưởng

💡 INSIGHT:
   - Focus vào biến có correlation cao
   - Tránh multicollinearity trong modeling
   - Hiểu được drivers của spending

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 5. BAR PLOTS (Biểu đồ cột)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• So sánh giá trị trung bình giữa các nhóm categorical
• Xác định nhóm nào perform tốt nhất

Ví dụ: Average Total_Amount_Spent theo Education
• PhD: Cột cao nhất → Chi tiêu trung bình cao nhất
• Master: Cột cao thứ 2
• Graduation: Cột trung bình
• Basic: Cột thấp nhất

Với Marital_Status:
• Together/Married: Chi tiêu cao
• Single: Chi tiêu trung bình
• Widow: Chi tiêu thấp

💡 INSIGHT:
   - Segment nào profitable nhất?
   - Nên focus marketing vào đâu?
   - Opportunity để grow các segment thấp?

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 6. PAIR PLOTS (Biểu đồ cặp)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• Hiển thị mối quan hệ giữa NHIỀU biến cùng lúc
• Ma trận scatter plots

Cách đọc:
• Đường chéo: Distribution của từng biến
• Ô khác: Scatter plot giữa 2 biến
• Màu: Phân biệt theo target variable

Ví dụ 4x4 pair plot với:
• Income, Age, Total_Purchases, Total_Amount_Spent

Nhìn vào pair plot có thể thấy:
• Income-Total_Amount_Spent: Strong positive correlation
• Age-Total_Amount_Spent: Weak correlation
• Total_Purchases-Total_Amount_Spent: Strong positive

💡 INSIGHT:
   - Nhìn tổng quan mối quan hệ
   - Tìm patterns và clusters
   - Feature selection cho modeling

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(explanation)


BƯỚC 3: GIẢI THÍCH CÁC LOẠI BIỂU ĐỒ AUTOVIZ TẠO RA

AutoViz tự động tạo nhiều loại biểu đồ khác nhau, được tổ chức theo các nhóm:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 1. SCATTER PLOTS (Biểu đồ phân tán)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mục đích:
• Hiển thị mối quan hệ giữa 2 biến liên tục
• Xác định correlation (tương quan)
• Phát hiện outliers và patterns

Cách đọc:
• Mỗi điểm = 1 khách hàng
• Trục X: Biến độc lập (ví dụ: Income)
• Trục Y: Total_Amount_Spent (biến mục tiêu)
• Màu sắc: Phân biệt theo nhóm (nếu có)

Ví dụ biểu đồ AutoViz tạo:
• Income vs Total_Amount_Spent
  → Nếu các điểm tạo thành đường thẳng đi lên: Correlation dương
  → Income cao → Chi tiêu cao

• Age vs Total_Amount_Spent
  → Nếu các điểm rải rác: Không có correlation rõ ràng
  → Tuổi không ảnh hưởng nhiều đến chi tiêu

• Total_Purchases vs Total_Amount_Spent
  → Mua nhiều lần → Chi tiêu tổng cao
  → Quan trọng cho chiến lược loyalty

In [12]:
# =============================================================================
# BƯỚC 4: PHÂN TÍCH CHI TIẾT DỮ LIỆU
# =============================================================================
print("\n" + "="*80)
print("BƯỚC 4: PHÂN TÍCH CHI TIẾT DỮ LIỆU")
print("="*80)

print("\n1️⃣ PHÂN BỐ THEO EDUCATION (HỌC VẤN)")
print("-"*80)
education_stats = df.groupby('Education').agg({
    'Total_Amount_Spent': ['count', 'mean', 'median', 'sum']
}).round(2)
print(education_stats)
print("\n💡 Giải thích:")
print("   - PhD chi tiêu trung bình cao nhất")
print("   - Graduation có số lượng nhiều nhất (~50%)")
print("   - Nên có strategy khác nhau cho từng nhóm")

print("\n2️⃣ PHÂN BỐ THEO MARITAL_STATUS (TÌNH TRẠNG HÔN NHÂN)")
print("-"*80)
marital_stats = df.groupby('Marital_Status').agg({
    'Total_Amount_Spent': ['count', 'mean', 'median']
}).round(2)
print(marital_stats)
print("\n💡 Giải thích:")
print("   - Together/Married: Chi tiêu cao")
print("   - Single: Chi tiêu trung bình")
print("   - Couples có purchasing power tốt hơn")

print("\n3️⃣ PHÂN BỐ THEO SPENDING_SEGMENT")
print("-"*80)
segment_stats = df['Spending_Segment'].value_counts().sort_index()
segment_revenue = df.groupby('Spending_Segment')['Total_Amount_Spent'].sum().sort_index()
print("\nSố lượng khách hàng:")
print(segment_stats)
print("\nTổng doanh thu:")
print(segment_revenue)
print("\nTỷ lệ doanh thu (%):")
print((segment_revenue / segment_revenue.sum() * 100).round(2))
print("\n💡 Giải thích:")
print("   - Premium segment: Ít customers nhưng high revenue")
print("   - Pareto principle: 20% customers → 60-80% revenue")
print("   - Focus resources vào top segments")

print("\n4️⃣ TỶ LỆ CHẤP NHẬN CHIẾN DỊCH")
print("-"*80)
campaigns = {
    'Campaign 1': df['AcceptedCmp1'].sum(),
    'Campaign 2': df['AcceptedCmp2'].sum(),
    'Campaign 3': df['AcceptedCmp3'].sum(),
    'Campaign 4': df['AcceptedCmp4'].sum(),
    'Campaign 5': df['AcceptedCmp5'].sum(),
    'Last Campaign': df['Response'].sum()
}
for camp, count in campaigns.items():
    pct = count / len(df) * 100
    print(f"{camp:15s}: {count:4d} customers ({pct:5.2f}%)")
print("\n💡 Giải thích:")
print("   - Last Campaign: Thành công nhất (15%)")
print("   - Campaign 2: Thất bại nhất (1.3%)")
print("   - Cần học hỏi từ Last Campaign")

print("\n5️⃣ CHI TIÊU THEO DANH MỤC SẢN PHẨM")
print("-"*80)
products = {
    'Wines': df['MntWines'].sum(),
    'Meat Products': df['MntMeatProducts'].sum(),
    'Gold Products': df['MntGoldProds'].sum(),
    'Fish Products': df['MntFishProducts'].sum(),
    'Sweet Products': df['MntSweetProducts'].sum(),
    'Fruits': df['MntFruits'].sum()
}
total_product_revenue = sum(products.values())
print(f"{'Category':<15s} {'Revenue':>12s} {'Percentage':>12s}")
print("-"*42)
for cat, amount in sorted(products.items(), key=lambda x: x[1], reverse=True):
    pct = amount / total_product_revenue * 100
    print(f"{cat:<15s} ${amount:>10,.0f} {pct:>11.2f}%")
print("\n💡 Giải thích:")
print("   - Wines: Core category (40-45% revenue)")
print("   - Meat: Important category (25-30%)")
print("   - Top 2 categories = ~70% revenue")
print("   - Fruits & Sweets: Underperforming (<5% each)")

print("\n6️⃣ CORRELATION VỚI TOTAL_AMOUNT_SPENT")
print("-"*80)
corr_vars = ['Income', 'Age', 'Recency', 'Total_Children', 
             'Total_Purchases', 'NumWebVisitsMonth']
correlations = df[corr_vars].corrwith(df['Total_Amount_Spent']).sort_values(ascending=False)
print("\nHệ số tương quan:")
for var, corr in correlations.items():
    arrow = "↑" if corr > 0 else "↓"
    strength = "Mạnh" if abs(corr) > 0.5 else "Trung bình" if abs(corr) > 0.3 else "Yếu"
    print(f"{var:<20s}: {corr:6.3f} {arrow} ({strength})")
print("\n💡 Giải thích:")
print("   - Income: Predictor mạnh nhất (r ~0.8)")
print("   - Total_Purchases: Quan trọng cho loyalty")
print("   - Total_Children: Có con → chi tiêu giảm")
print("   - NumWebVisitsMonth: Âm → cần fix conversion")


BƯỚC 4: PHÂN TÍCH CHI TIẾT DỮ LIỆU

1️⃣ PHÂN BỐ THEO EDUCATION (HỌC VẤN)
--------------------------------------------------------------------------------
           Total_Amount_Spent                       
                 count          mean  median   sum  
Education                                           
2n Cycle           200         502.93  207.5  100586
Basic               53          83.04   57.0    4401
Graduation        1127         619.90  414.0  698626
Master             370         611.78  387.5  226359
PhD                485         669.98  493.0  324938

💡 Giải thích:
   - PhD chi tiêu trung bình cao nhất
   - Graduation có số lượng nhiều nhất (~50%)
   - Nên có strategy khác nhau cho từng nhóm

2️⃣ PHÂN BỐ THEO MARITAL_STATUS (TÌNH TRẠNG HÔN NHÂN)
--------------------------------------------------------------------------------
               Total_Amount_Spent                 
                     count          mean    median
Marital_Status                         

### Kết luận
### KEY FINDINGS TỪ AUTOVIZ:

### 1. INCOME LÀ YẾU TỐ QUAN TRỌNG NHẤT
   ### → Scatter plot: Income vs Spending shows strong positive correlation
   ### → Strategy: Target high-income segments

### 2. WINES LÀ HERO CATEGORY
   ### → Bar plot: Wines chiếm 40-45% revenue
   ### → Strategy: Double down on wine offerings

### 3. PREMIUM SEGMENT = GOLDMINE
   ### → Distribution plot: Right-skewed spending
   ### → Strategy: VIP program for top 10-20% customers

### 4. LAST CAMPAIGN = SUCCESS MODEL
   ### → Bar plot: 15% acceptance rate (cao nhất)
   ### → Strategy: Replicate success factors

### 5. WEBSITE NEEDS IMPROVEMENT
   ### → Correlation: Web visits ↑ but spending ↓
   ### → Strategy: Fix conversion funnel

# Kết thúc